In [1]:
import pandas as pd
from pathlib import Path
import json
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import numpy as np
from matplotlib import colormaps
import nibabel as nib
from scipy import ndimage
import csv

from mri_data import file_manager as fm

In [2]:
hipsthomas_root = Path("/media/smbshare/srs-9/hipsthomas")
dataproc_root = Path("/media/smbshare/srs-9/thalamus_project/data")
data_file_dir = Path("/home/srs-9/Projects/ms_mri/analysis/thalamus/data0")

In [18]:
def load_thomas(root, sub, ses):
    thomas_dir = root / f"sub{sub}-{ses}"
    thomL_file = thomas_dir / "left/thomasfull_L.nii.gz"
    thomL_img = nib.load(thomL_file)
    thomR_file = thomas_dir / "right/thomasfull_R.nii.gz"
    thomR_img = nib.load(thomR_file)
    return thomL_img, thomR_img

In [22]:
thomL_img,_ = load_thomas(hipsthomas_root, 1001, 20170215)
ventricle_sdt_path = "/media/srs-9/Data/srs-9/hips-thomas/sub1001-20170215/aseg-ventricle-sdt.nii.gz"
ventricle_sdt = nib.load(ventricle_sdt_path).get_fdata()

thom = thomL_img.get_fdata()
thom_inds = np.unique(thom)
thom_inds = thom_inds[thom_inds > 0]
dists = {}
for ind in thom_inds:
    struct_pts = thom.copy()
    struct_pts[thom!=ind] = 0
    struct_pts[thom==ind] = 1
    centroid = ndimage.center_of_mass(struct_pts)
    centroid_round = [int(cent) for cent in centroid]
    dists[int(ind)] = ventricle_sdt[*centroid_round]

In [27]:
print(dists)

{2: 4.079217910766602, 4: 6.144927978515625, 5: 10.881211280822754, 6: 9.363773345947266, 7: 10.762900352478027, 8: 8.541664123535156, 9: 18.33031463623047, 10: 15.839193344116211, 11: 12.470782279968262, 12: 9.329524040222168, 13: 13.410449981689453, 14: 10.055845260620117, 26: 6.499259948730469, 27: 5.122552394866943, 28: 20.83097267150879, 29: 10.851828575134277, 30: 11.482226371765137, 31: 15.450724601745605, 32: 18.434751510620117}


In [30]:
data_file = "/home/srs-9/Projects/ms_mri/choroid_thalamus_project/sub1001-centroid-ventricle-sdt.csv"
with open(data_file, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["ind", "dist"])
    for k,v in dists.items():
        print(k, v)
        writer.writerow([k, v])

2 4.079217910766602
4 6.144927978515625
5 10.881211280822754
6 9.363773345947266
7 10.762900352478027
8 8.541664123535156
9 18.33031463623047
10 15.839193344116211
11 12.470782279968262
12 9.329524040222168
13 13.410449981689453
14 10.055845260620117
26 6.499259948730469
27 5.122552394866943
28 20.83097267150879
29 10.851828575134277
30 11.482226371765137
31 15.450724601745605
32 18.434751510620117


---

In [4]:
drive_root = fm.get_drive_root()
dataroot = drive_root / "3Tpioneer_bids"
with open(dataroot / "subject-sessions-longit.json", 'r') as f:
    subject_sessions = json.load(f)

df = pd.read_csv("/home/srs-9/Projects/ms_mri/data/clinical_data_processed.csv", index_col="subid")[['dzdur', 'ms_type', 'dz_type5']]

In [5]:
date_format = "%Y%m%d"
deltas = []
for k, v in subject_sessions.items():
    sessions = sorted(v)
    time1 = datetime.strptime(str(sessions[0]), date_format)
    time2 = datetime.strptime(str(sessions[-1]), date_format)
    delta = (time2 - time1).days
    deltas.append((time2 - time1).days)
    df.loc[int(k), 'delta'] = delta / 365

deltas.sort(reverse=True)
deltas = np.array(deltas)

In [ ]:
check = df[df['dz_type5'].isin(["PMS"])].sort_values(by='dzdur')
check2 = check[check['delta'] > 3]

rms_check = df[df['dz_type5'].isin(["RMS"])]
rms_check2 = rms_check[(rms_check['delta'] > 4) & (rms_check['dzdur'] < 5)]

In [28]:
subjects = check2.index.tolist() + rms_check2.index.tolist()
with open("subjects_to_process.txt", 'w') as f:
    for sub in subjects:
        f.write(str(sub) + "\n")

In [10]:
check.loc[1198, 'ms_type']

'PPMS'

In [12]:
subject_sessions['1131']

[20211229, 20201218, 20191221, 20180105, 20181220, 20161209]